# Section 4 — Universal Functions (ufuncs): Internals and Custom Creation

Universal functions (ufuncs) are the backbone of NumPy’s fast element-wise operations. They handle broadcasting, type coercion, and loop vectorization under the hood — allowing you to perform millions of scalar operations efficiently in pure C.  

In this section, we’ll explore:
- What ufuncs are and how they differ from ordinary Python functions
- Their internal mechanisms (loops, type signatures, and broadcasting)
- How to create your own ufuncs for specialized math operations
- Performance comparisons and real-world usage

## 4.1 What Are ufuncs?

A **ufunc** (universal function) is a *vectorized wrapper around a C loop* that applies an operation element-wise to NumPy arrays.  
Every arithmetic operation (`+`, `-`, `*`, `/`, etc.) in NumPy is backed by a ufunc.

They are designed for speed, automatic broadcasting, and flexible type handling. Let’s see them in action.

In [ ]:
import numpy as np

# Two arrays of compatible shapes
a = np.array([1, 2, 3, 4])
b = np.array([10, 20, 30, 40])

# Using NumPy's add ufunc (same as a + b)
result = np.add(a, b)
print("Result using np.add:", result)

# Demonstrating broadcasting with ufuncs
result_broadcast = np.add(a, 100)
print("Broadcasted addition:", result_broadcast)

NumPy automatically broadcasts scalar or smaller-dimensioned arrays to match the shapes of others.  
Behind the scenes, each ufunc executes a compiled C loop for every data type combination (float32, int64, etc.), and dispatches to the correct one at runtime.

## 4.2 Inspecting ufunc Properties

Each ufunc has rich metadata you can inspect, such as the number of inputs/outputs and its available type signatures. This helps you understand how NumPy efficiently handles data types and operations internally.

In [ ]:
# Exploring ufunc attributes
uf = np.add
print("Name:", uf.__name__)
print("Number of inputs:", uf.nin)
print("Number of outputs:", uf.nout)
print("Type signatures:")
for sig in uf.types[:5]:  # show first few
    print(' ', sig)

Every entry in `uf.types` represents a data type combination, like `'dd->d'` for double + double → double.  
This design makes ufuncs *extremely flexible* and type-safe, without needing Python-level conditionals.

## 4.3 Creating Custom ufuncs

There are several ways to create your own ufuncs:
- **`np.frompyfunc`**: Converts a pure Python function to a generic ufunc (object type).
- **`np.vectorize`**: A convenience wrapper that provides a ufunc-like interface but with Python loops (slower).
- **Numba/Cython-based ufuncs**: For compiled performance.

Let's explore both `frompyfunc` and `vectorize`.

In [ ]:
# Example: custom safe division function
def safe_divide(x, y):
    return x / y if y != 0 else np.nan

# Creating a ufunc from Python function
safe_divide_ufunc = np.frompyfunc(safe_divide, 2, 1)

a = np.array([10, 20, 30])
b = np.array([2, 0, 5])

print("Result from frompyfunc:", safe_divide_ufunc(a, b))

Note that `frompyfunc` always returns an array of type `object` — this provides flexibility but at the cost of speed.  
For numeric efficiency, use `np.vectorize`, which attempts to infer data types and returns native NumPy arrays.

In [ ]:
# Using np.vectorize for numeric output
safe_divide_vec = np.vectorize(safe_divide)
print("Result from vectorize:", safe_divide_vec(a, b))

## 4.4 Under the Hood: How ufuncs Work

Under the hood, every ufunc performs the following sequence:
1. **Input coercion:** Convert input arrays to compatible dtypes.
2. **Loop dispatch:** Select the correct inner C loop based on dtype signatures.
3. **Broadcasting:** Expand smaller shapes to match larger ones.
4. **Iteration:** Iterate over elements at C speed, performing the operation.
5. **Output handling:** Allocate or reuse the result array, cast to proper dtype.

This entire process occurs without Python overhead, which is why NumPy’s ufuncs are orders of magnitude faster than Python loops.

## 4.5 Best Practices & Pitfalls

**✅ Best Practices:**
- Prefer native ufuncs over Python loops for element-wise math.
- Use `out=` parameter in ufuncs for in-place operations (saves memory).
- Use `where=` to control which elements are computed.

**⚠️ Pitfalls:**
- `np.vectorize` and `np.frompyfunc` are convenient but slower than native ufuncs.
- Avoid mixing Python scalars and arrays with different dtypes.
- Be cautious with divisions, modulo, or power operations on integer arrays — promote to float when needed.

In [ ]:
# Example of in-place ufunc operation
x = np.array([1, 2, 3, 4], dtype=float)
np.multiply(x, 10, out=x)  # In-place operation
print("In-place result:", x)

# Using where parameter
y = np.array([1, -2, 3, -4])
res = np.abs(y, where=(y<0))
print("Conditional abs:", res)

## 🧩 Challenge Exercise

**Task:** Create a custom ufunc that computes the *harmonic mean* of two arrays:

\[ H(x, y) = \frac{2xy}{x + y} \]

Requirements:
- Implement using both `np.frompyfunc` and `np.vectorize`.
- Handle division-by-zero gracefully.
- Compare performance and dtype behavior between the two implementations.

_(No solution provided here — try it yourself!)_

---
# --- End of Section 4 — Continue to Section 5 ---
In the next section, we’ll dive into **Performance Tuning and Vectorization Strategies**, exploring how to make NumPy code even faster by optimizing memory layout and computation paths.